In [1]:
import pandas as pd
import numpy as np
from math import isnan

In [2]:
with open('PLmatches.csv') as open_file:
    df = pd.read_csv(open_file, index_col=0)

In [3]:
def get_prior_matches(team_name, season_num, match_timestamp, num_matches=38):
    all_prior_matches = df[((df['hometeam_name'] == team_name)|(df['awayteam_name'] == team_name))
                           #&(df['season_num'] == season_num)
                          &(df['match_timestamp'] < match_timestamp)].sort_values(by='match_timestamp')
    
    return all_prior_matches[-num_matches:]

In [4]:
def get_mean_value_by_team(data, team_name, col_name):
    values = []
    for row_index in list(data.index.values):
        if data.loc[row_index, 'hometeam_name'] != team_name and data.loc[row_index, 'awayteam_name'] != team_name:
            continue
        team_loc = 'home' if data.loc[row_index, 'hometeam_name'] == team_name else 'away'
        values.append(data.loc[row_index, '_'.join([team_loc, col_name])])
    
    return np.mean(values)

def get_values_by_team(data, team_name, col_name):
    values = []
    for row_index in data.index.values:
        if data.loc[row_index, 'hometeam_name'] != team_name and data.loc[row_index, 'awayteam_name'] != team_name:
            continue
        team_loc = 'home' if data.loc[row_index, 'hometeam_name'] == team_name else 'away'
        values.append(data.loc[row_index, '_'.join([team_loc, col_name])])
    
    return values

In [10]:
def assign_home_leaguepts_gained(row):
    score_diff = row['home_goals'] - row['away_goals']
    if score_diff > 0:
        return 3
    elif score_diff < 0:
        return 0
    else: return 1

def assign_away_leaguepts_gained(row):
    score_diff = row['away_goals'] - row['home_goals']
    if score_diff > 0:
        return 3
    elif score_diff < 0:
        return 0
    else: return 1

In [12]:
df['home_leaguepts_gained'] = df.apply(assign_home_leaguepts_gained, axis=1)
df['away_leaguepts_gained'] = df.apply(assign_away_leaguepts_gained, axis=1)

In [14]:
def get_last3_home_leaguepts(row):
    team_name = row['hometeam_name']
    season_num = row['season_num']
    cur_timestamp = row['match_timestamp']
    
    df_lastgame = get_prior_matches(team_name, season_num, cur_timestamp, 3)
    
    return get_mean_value_by_team(df_lastgame, team_name, 'leaguepts_gained')

def get_last3_away_leaguepts(row):
    team_name = row['awayteam_name']
    season_num = row['season_num']
    cur_timestamp = row['match_timestamp']
    
    df_lastgame = get_prior_matches(team_name, season_num, cur_timestamp, 3)
    
    return get_mean_value_by_team(df_lastgame, team_name, 'leaguepts_gained')

In [15]:
df['home_last3_leaguepts'] = df.apply(get_last3_home_leaguepts, axis=1)

/home/dlahtou/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/dlahtou/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [16]:
df['away_last3_leaguepts'] = df.apply(get_last3_away_leaguepts, axis=1)

/home/dlahtou/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/dlahtou/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [17]:
df[((df['hometeam_name'] == 'Aston Villa')|(df['awayteam_name'] == 'Aston Villa'))&(df['season_num'] == 4)]

,hometeam_name,awayteam_name,home_goals,away_goals,match_date,match_timestamp,match_referee,match_stadium,match_attendance,home_possession,...,away_Ndec,away_Wdec,away_distance,precip_prob,temp_high,week_power,home_last3_leaguepts,away_last3_leaguepts,home_leaguepts_gained,away_leaguepts_gained
6707,Aston Villa,Wigan Athletic,0,2,Sat 15 Aug 2009,1250344800000,Mark Clattenburg,"Villa Park, Birmingham",35578.0,47.0,...,53.5478,-2.6539,126.578835,NaN,NaN,10,1.333333,1.000000,0,3
6732,Liverpool,Aston Villa,1,3,Mon 24 Aug 2009,1251140400000,Martin Atkinson,"Anfield, Liverpool",43667.0,67.4,...,52.5092,-1.8847,125.477915,NaN,NaN,8,2.000000,1.333333,0,3
6740,Aston Villa,Fulham,2,0,Sun 30 Aug 2009,1251644400000,Steve Bennett,"Villa Park, Birmingham",32917.0,44.1,...,51.4750,-0.2217,162.135406,NaN,NaN,7,2.000000,1.000000,3,0
6751,Birmingham City,Aston Villa,0,1,Sun 13 Sep 2009,1252839600000,Howard Webb,"St Andrews, Birmingham",25196.0,50.3,...,52.5092,-1.8847,3.892590,NaN,NaN,6,1.333333,2.000000,0,3
6754,Aston Villa,Portsmouth,2,0,Sat 19 Sep 2009,1253368800000,Stuart Attwell,"Villa Park, Birmingham",35979.0,49.0,...,50.7964,-1.0639,198.850232,NaN,NaN,5,3.000000,0.000000,3,0
6764,Blackburn Rovers,Aston Villa,2,1,Sat 26 Sep 2009,1253973600000,Mark Clattenburg,"Ewood Park, Blackburn",25172.0,47.5,...,52.5092,-1.8847,141.610560,NaN,NaN,4,1.333333,3.000000,3,0
6782,Aston Villa,Manchester City,1,1,Mon 5 Oct 2009,1254769200000,Mike Dean,"Villa Park, Birmingham",37924.0,43.8,...,53.4831,-2.2003,110.433417,NaN,NaN,3,2.000000,2.000000,1,1
6784,Aston Villa,Chelsea,2,1,Sat 17 Oct 2009,1255779900000,Kevin Friend,"Villa Park, Birmingham",39047.0,34.5,...,51.4817,-0.1911,163.093243,NaN,NaN,2,1.333333,2.000000,3,0
6798,Wolverhampton Wanderers,Aston Villa,1,1,Sat 24 Oct 2009,1256384700000,Peter Walton,"Molineux Stadium, Wolverhampton",28734.0,48.4,...,52.5092,-1.8847,18.947198,NaN,NaN,1,0.333333,1.333333,1,1
6806,Everton,Aston Villa,1,1,Sat 31 Oct 2009,1257001200000,Lee Probert,"Goodison Park, Liverpool",36648.0,47.7,...,52.5092,-1.8847,126.427711,NaN,NaN,0,0.666667,1.666667,1,1


In [18]:
with open('PLmatches.csv', 'w') as open_file:
    df.to_csv(open_file)